In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime



In [63]:
def get_major_news(ticker):
    news = ticker.news
    data = []
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        # Convertir le timestamp Unix en format lisible
        publish_time = datetime.fromtimestamp(item.get('providerPublishTime')).strftime('%Y-%m-%d %H:%M:%S')
        
        # Ajouter les informations dans un dictionnaire
        data.append({
            'Title': title,
            'Link': link,
            'Source': source,
            'Publish Time': publish_time
        })
    
    # Convertir la liste en DataFrame
    df = pd.DataFrame(data)
    return df

In [64]:
news = get_major_news(ticker)

In [67]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire le texte principal de l'article
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraction des paragraphes de l'article
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    return article_text

In [68]:
extract_article_text(news["Link"][0])

'We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists. (Bloomberg) -- Berkshire Hathaway Inc.’s continued sales of Apple Inc. shares in the third quarter left the conglomerate’s stake at a fraction of its size at the start of the year. Most Read from Bloomberg Istanbul Tries Free Public Transit to Help Job Seekers The Answer To Making Cities More Family-Friendly? Courtyards Is Denver’s Big Bet on E-Bikes Paying Off? In Warsaw, Falling Road Deaths Signal a Traffic Safety Turnaround Chicago Mayor Seeks Property Tax Hike, Breaking Campaign Vow Berkshire cut its holdings in the iPhone maker by roughly 25% in the period, after slashing it by almost half in the second quarter. Apple’s shares gained 10.6% in the period ended Sept. 30. Warren Buffett hinted at Berkshire’s annual meeting in May that the sales of Apple in the first quarter were partly motivated by tax implications,

In [69]:
import logging
import boto3
from botocore.exceptions import ClientError

In [70]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [71]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [72]:

bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [92]:

def analyze_subject_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 100,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Donne uniquement sujet principal de cet article ainsi que le sentiment du text associé sans explicatioN: \"{text}\""}
                ]
            })
        )
        
        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        subject_analysis = content[0]["text"] if content else "Pas de sujet détecté"
        return subject_analysis
    
    except Exception as e:
        print("Erreur d'analyse du sujet :", e)
        return None



In [89]:

def analyze_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)
        return None

In [90]:
url = "https://finance.yahoo.com/news/apple-intelligence-products-could-apple-161448615.html"
article_text = extract_article_text(url)

In [93]:
sujet_principal = analyze_subject_sentiment(article_text)
print("Sujet Principal de l'Article :", sujet_principal)

Sujet Principal de l'Article : Le sujet principal de cet article semble être les perspectives d'Apple dans la course à l'intelligence artificielle (IA) et le lancement potentiel d'un appareil Apple Intelligence. Le sentiment général du texte est plutôt neutre, présentant à la fois les défis auxquels Apple est confronté dans ce domaine ainsi que les opportunités futures avec le lancement de nouveaux produits IA.


In [94]:

# Fonction principale pour obtenir les informations d'actualité avec sujet et sentiment
def get_news_with_sentiment(ticker):
    news = ticker.news
    sentiment_output = ''
    
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        
        # Extraire le texte de l'article
        article_text = extract_article_text(link)
        
        # Analyser le sujet et le sentiment
        subject_sentiment = analyze_subject_sentiment(article_text)
        
        sentiment_output += title + subject_sentiment
        

    return sentiment_output

In [95]:
get_news_with_sentiment(ticker)

'Buffett Keeps Cutting Apple Stake, With Value Down 60% This YearSujet principal : L\'article traite principalement de la vente continue par Berkshire Hathaway d\'actions d\'Apple au troisième trimestre, réduisant considérablement sa participation dans la société technologique.\n\nSentiment : Le ton de l\'article est plutôt neutre et factuel, présentant des informations et des chiffres sur les activités de Berkshire Hathaway sans émettre3 Things Apple Doesn\'t Want to Talk AboutSujet principal: Problème technique en cours de résolution\n\nSentiment: RassurantBig Pension Sold Palantir, Apple, and Nvidia Stock. It Bought AMD.Sujet principal: Problème technique en cours de résolution\n\nSentiment: RassurantApple Intelligence Products Could Make Up For Apple’s (AAPL) iPhone WeaknessLe sujet principal de cet article semble être les perspectives d\'Apple dans la course à l\'intelligence artificielle et le lancement potentiel d\'un appareil "Apple Intelligence".\n\nLe sentiment général du tex

## SP500

In [ ]:
def get_sp500_list():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    sp500_table = pd.read_html(url, header=0)[0]
    
    sp500_table = sp500_table.drop(columns=['Headquarters Location', 'Date added','Founded', 'CIK','GICS Sub-Industry','GICS Sector'])
    
    return sp500_table

In [8]:
sp500_table =get_sp500_list()

In [9]:
sp500_table

,Symbol,Security
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott Laboratories
3,ABBV,AbbVie
4,ACN,Accenture
...,...,...
498,XYL,Xylem Inc.
499,YUM,Yum! Brands
500,ZBRA,Zebra Technologies
501,ZBH,Zimmer Biomet


In [16]:
from rapidfuzz import process, fuzz

def get_ticker_from_name(name, df, threshold=50):
    match = process.extractOne(name, df["Security"], scorer=fuzz.token_sort_ratio)
    if match and match[1] >= threshold:  # Seuil pour déterminer la qualité du match
        symbol = df.loc[df["Security"] == match[0], "Symbol"].values[0]
        return symbol
    else:
        return "Ticker non trouvé"

# Exemple d'utilisation
ticker = get_ticker_from_name("3M", sp500_table)
print(f"Le ticker pour 'Apple' est : {ticker}")

Le ticker pour 'Apple' est : MMM


## Macros

In [52]:
def caption_summary(text,max_token):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_token,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Tu es un macro économiste, rédige un résumé des points clés  macro économique des différents sujets, avec détails (ex chiffres) : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)

In [53]:
caption_summary("https://finance.yahoo.com/m/0b9780b5-16af-3ee9-b4a1-926fcb7e6826/the-stock-market-is-doing.html",150)

"Voici un résumé des principaux points macroéconomiques abordés dans l'article, accompagné de détails chiffrés :\n\nCroissance économique :\n- Le PIB réel des États-Unis a progressé de 2,6 % au quatrième trimestre 2022, selon les estimations initiales.\n- La croissance annuelle du PIB réel pour 2022 devrait s'établir autour de 2,1 %, en baisse par rapport aux 5,9 % enregistrés en 2021.\n\nInflation :\n- L'inflation mesurée par l'in"

In [54]:
import yfinance as yf
import pandas as pd


tickers = {
    "^GSPC": "S&P 500 - Indice Boursier",
    "DX-Y.NYB": "Indice du Dollar Américain",
    "^TNX": "Taux des Obligations à 10 ans",
    "GC=F": "Prix de l'Or",
    "CL=F": "Prix du Pétrole (WTI)",
    "^VIX": "VIX - Volatilité des Marchés"
}


In [ ]:
news_data = []

output_text = ""
for ticker_symbol, description in tickers.items():
    ticker = yf.Ticker(ticker_symbol)
    news_items = ticker.news
    
    for item in news_items[:2]:
        
        text = caption_summary(item["link"],150)
        output_text += text

In [58]:
caption_summary(output_text,max_token=300)

"Voici un résumé des points clés macroéconomiques abordés dans l'article, avec des détails chiffrés :\n\nContexte économique actuel :\n- L'économie américaine connaît une période de ralentissement, avec un taux de croissance annualisé du PIB de seulement 1,1% au premier trimestre 2023. \n- Cependant, le marché boursier (S&P 500) affiche une performance positive de 8,6% depuis le début de l'année.\n\nMarché du travail :\n- Le taux de chômage est resté stable à 3,6% en avril 2023.\n- Seulement 253 000 nouveaux emplois ont été créés en avril, bien en deçà des attentes des économistes qui tablaient sur 180 000 nouveaux emplois. Ce ralentissement contraste avec les solides créations d'emplois des mois précédents (472 000 en janvier et 326 000 en février).\n\nInflation :\n- L'inflation annuelle des prix à la consommation s'est établie à 4,9% en avril, en léger ralentissement par rapport à mars (5%).\n- Cependant"